In [ ]:
# sklearn linear regression

In [420]:
import gzip
import numpy
from collections import defaultdict

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [421]:
data = []
for l in readGz("assignment1/train.json.gz"):
    data.append(l)

In [422]:
import random

# Randomly re-shuffle
# numpy.random.shuffle(data)
data_train = data[:100000]
data_valid = data[100000:]

In [423]:
import math 

def inner(x,y):
    return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
    return 1.0 / (1 + exp(-x))

In [443]:
def feature(datum):
    feat= [1]
    feat.append(sigmoid(datum['helpful']['outOf']))
    feat.append(len(datum['reviewText'].split()))  
    feat.append(datum['rating'] * 1.0 / 5.0)
#   categoryID    
    if (datum['categoryID'] == 0):
        for i in [1,0,0,0,0]:
            feat.append(i)
    elif (datum['categoryID'] == 1):
        for i in [0,1,0,0,0]:
            feat.append(i)
    elif (datum['categoryID'] == 2):
        for i in [0,0,1,0,0]:
            feat.append(i)
    elif (datum['categoryID'] == 3):
        for i in [0,0,0,1,0]:
            feat.append(i)
    elif (datum['categoryID'] == 4):
        for i in [0,0,0,0,1]:
            feat.append(i)
            
    return feat

In [444]:
X = [feature(d) for d in data if d['helpful']['outOf'] > 0]
y = [d['helpful']['nHelpful'] * 1.0 / d['helpful']['outOf'] for d in data if d['helpful']['outOf'] > 0]

In [445]:
X_1 = [feature(d) for d in data if d['helpful']['outOf'] > 2]
X_2 = [feature(d) for d in data if d['helpful']['outOf'] > 0 & d['helpful']['outOf'] < 3]
y_1 = [d['helpful']['nHelpful'] * 1.0 / d['helpful']['outOf'] for d in data if d['helpful']['outOf'] > 2]
y_2 = [d['helpful']['nHelpful'] * 1.0 / d['helpful']['outOf'] for d in data if d['helpful']['outOf'] > 0 & d['helpful']['outOf'] < 3]

In [446]:
import numpy as np
from sklearn import linear_model

regr1 = linear_model.LinearRegression()
regr2 = linear_model.LinearRegression()
regr1.fit(X_1, y_1)
regr2.fit(X_2, y_2)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [447]:
X_v_1 = [feature(d) for d in data_valid if d['helpful']['outOf'] > 2]
out_1 = [d['helpful']['outOf'] for d in data_valid if d['helpful']['outOf'] > 2]
y_v_1 = [d['helpful']['nHelpful'] for d in data_valid if d['helpful']['outOf'] > 2]

X_v_2 = [feature(d) for d in data_valid if d['helpful']['outOf'] > 0 & d['helpful']['outOf'] < 3]
out_2 = [d['helpful']['outOf'] for d in data_valid if d['helpful']['outOf'] > 0 & d['helpful']['outOf'] < 3]
y_v_2 = [d['helpful']['nHelpful'] for d in data_valid if d['helpful']['outOf'] > 0 & d['helpful']['outOf'] < 3]


In [448]:
predict1 = regr1.predict(X_v_1)
predict2 = regr2.predict(X_v_2)

In [449]:
MAE = 0
for i in range(len(predict1)):
    res = round(predict1[i] * out_1[i])
    MAE += math.fabs(res - y_v_1[i])
    
for i in range(len(predict2)):
    res = round(predict2[i] * out_2[i])
    MAE += math.fabs(res - y_v_2[i])
    
MAE /= len(data_valid)
print "MAE: " + str(MAE)

MAE: 0.31941


In [450]:
from sklearn.ensemble import GradientBoostingRegressor as gdr
regressor1 = gdr(loss='ls')
regressor1.fit(X_1, y_1)
predict3 = regressor1.predict(X_v_1)

regressor2 = gdr(loss='ls')
regressor2.fit(X_2, y_2)
predict4 = regressor2.predict(X_v_2)

In [451]:
MAE1 = 0
for i in range(len(predict1)):
    res = round(predict1[i] * out_1[i])
    MAE1 += math.fabs(res - y_v_1[i])
    
for i in range(len(predict2)):
    res = round(predict2[i] * out_2[i])
    MAE1 += math.fabs(res - y_v_2[i])
    
MAE1 /= len(data_valid)
print "MAE: " + str(MAE1)

MAE: 0.31941


In [ ]:
MAE: 0.17423

In [312]:
# MAE
MAE = 0
for d in data_valid:
    if d['helpful']['outOf'] > 0:
        feat = feature(d)
        predict = round(d['helpful']['outOf'] * inner(feat, theta1))
        MAE += math.fabs(predict - d['helpful']['nHelpful'])

print("Validation MAE = " + str(MAE / len(data_valid)))

Validation MAE = 0.20251


In [ ]:
MAE = 0.17987 sigmoid(outOf)

In [224]:
# Kaggle
reviews = {}
for l in readGz("assignment1/test_Helpful.json.gz"):
    user,item = l['reviewerID'], l['itemID']
    key = user + ' ' + item
    reviews[key] = {}
    reviews[key]["outOf"] = l["helpful"]["outOf"]
    
predictions = open("assignment1/predictions_Helpful.txt", 'w')

for l in open("assignment1/pairs_Helpful.txt"): 
    if l.startswith("userID"):
        # first line
        predictions.write(l)
        continue
    
    u, i, outOf = l.strip().split('-')
    outOf = int(outOf)
    key = u + ' ' + i
    feat = [1, sigmoid(reviews[key]["outOf"])]
    predict = round(outOf * inner(feat, theta))
    predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(predict)+ '\n')

predictions.close ()